In [ ]:
import pandas as pd
from pathlib import Path
from scipy.ndimage import gaussian_filter
import utils as utl

In [ ]:
parent_dir = Path(r'\\mpfi.org\public\sb-lab\Nino_2P_for_Salil\for_Nico\stop1_imaging\stop1-GCaMP6f-tdTomato_VNC\fed')

p_out = parent_dir / 'smth4_perc10_winsize50'

# merge all trials and flies
p_df = p_out / 'all_data_r12.parquet'

# create plot folder
p_plot = p_out / 'custom_plots_r12_nozscore'
p_plot.mkdir(exist_ok=True)

# read data from disk
df = pd.read_parquet(p_df)
df = df.fillna(0) # TODO workaround because of missing behavior

# optional: remove ROI 7, 8, 9
df = df.drop(columns=['z_roi_7', 'z_roi_8', 'z_roi_9'])
df = df.drop(columns=['roi_7', 'roi_8', 'roi_9'])

# frequencies used for imaging, ball velocities, and behavior
f_ca, f_ball, f_beh = 2, 50, 200

# plot aligned data
dt = 5 # time in s before and after behavior event
s = 0.25 # smoothing window for velocity [in s] 

# smooth velocity
df_ = df.copy()
df_.loc[:, ['ball_x', 'ball_y', 'ball_z']] = gaussian_filter(df_.loc[:, ['ball_x', 'ball_y', 'ball_z']].values, (s * f_beh, 0))

# cycle through all behavoirs
cols = [c for c in df_.columns if c.startswith('beh_')]
for col in cols:

    # align to behavior
    df_al = utl.align2events(df_, col, f_beh, dt, use_z_roi=False)

    utl.plot_aligned(df_al, ylims_roi=(.19, .35), ylims_ball=(-.35, 1.8), path=p_plot / f'aligned_to_{col}.svg')